1. 합치기  
2. depth값을 10*10 box에서 구함(segmentation 적용 방식은 고민해봐야할듯)
3. 여러 모델들 실험하다보니 변수명(Facenet, Mobilefacenet 등..)이 이상한 것이 있는데, 모두 Arcface임

In [ ]:
#!/usr/bin/env python
import time
import mediapipe as mp
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from deep_sort_realtime.deepsort_tracker import DeepSort
import insightface
from insightface.app import FaceAnalysis

import rospy
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
from ultralytics import YOLO
from geometry_msgs.msg import Twist

# ROS Initialization
rospy.init_node('kobuki_follow_person', anonymous=True)
cmd_vel_pub = rospy.Publisher('/mobile_base/commands/velocity', Twist, queue_size=10)

# PID Parameters
TARGET_DISTANCE = 1.0  # Target distance in meters
Kp_linear = 0.4   # Proportional gain for linear velocity
Ki_linear = 0.01  # Integral gain for linear velocity
Kd_linear = 0.1   # Derivative gain for linear velocity

Kp_angular = 0.4   # Proportional gain for angular velocity
Ki_angular = 0.01  # Integral gain for angular velocity
Kd_angular = 0.1   # Derivative gain for angular velocity

prev_error_linear = 0
integral_linear = 0
prev_error_angular = 0
integral_angular = 0

############################
# (A) Arcface (InsightFace) 준비
############################
import insightface
from insightface.app import FaceAnalysis

app = FaceAnalysis(name="buffalo_sc")  # "buffalo_sc"
app.prepare(ctx_id=0, det_size=(640,640))  # GPU라면 ctx_id=0, CPU는 -1

def get_face_embedding(image_bgr):
    if image_bgr is None or image_bgr.size==0:
        return None
    faces = app.get(image_bgr)
    if len(faces)==0:
        return None
    return faces[0].embedding  # shape 예: (128,)

def is_my_face(face_embedding, my_embedding, threshold=0.4):
    sim = cosine_similarity([face_embedding], [my_embedding])[0][0]
    return (sim > threshold), sim

############################
# (B) 모델/함수 초기화
############################
warnings.filterwarnings("ignore", category=FutureWarning)  #경고제거
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO("/home/dev/runs/detect/train55/weights/epoch180.pt").to(device)
model_seg = YOLO("yolov8n-seg.pt").to(device)

#임베딩 가져오기
my_face_embedding = np.load("my_face_embedding.npy")  # (128,) or (512,) etc.
print("Camera initializing with", device, '...')

############################
# (C) Mediapipe Pose 등 동일
############################
mp_pose = mp.solutions.pose
pose_danger = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

mp_drawing = mp.solutions.drawing_utils
LEFT_SHOULDER = 11
RIGHT_SHOULDER= 12
LEFT_WRIST   = 15
RIGHT_WRIST  = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    return wrist_y < (shoulder_y - threshold)

def boxes_overlap(boxA, boxB):
    (x1A, y1A, x2A, y2A) = boxA
    (x1B, y1B, x2B, y2B) = boxB
    overlap_x = not (x2A < x1B or x2B < x1A)
    overlap_y = not (y2A < y1B or y2B < y1A)
    return overlap_x and overlap_y

############################
# (D) DeepSORT
############################
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    embedder='mobilenet',
    half=True,
    embedder_gpu=True
)

############################
# (E) 기타 변수 설정
############################
dangerous_ids = set()
track_me_status = {}
track_arcface_count= {}
MAX_ARCFACE_FRAMES= 20
sim = 0

window_name = "DeepSORT + YOLO(SEG) + MobileFaceNet + Pose"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960,720)

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)


def compute_pid(error, prev_error, integral, Kp, Ki, Kd):
    """ Compute PID velocity command. """
    integral += error
    derivative = error - prev_error
    prev_error = error
    return (Kp * error) + (Ki * integral) + (Kd * derivative), prev_error, integral

def get_center_window_distance(depth_frame, x1, y1, x2, y2, window_size=30):
    """ Get the average distance of a 10x10 window at the center of the bounding box. """
    # Calculate the center of the bounding box
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    
    # Define the window around the center
    half_window = window_size // 2
    window_x1 = max(0, cx - half_window)
    window_y1 = max(0, cy - half_window)
    window_x2 = min(depth_frame.get_width(), cx + half_window)
    window_y2 = min(depth_frame.get_height(), cy + half_window)
    
    # Get distances for pixels in the 10x10 window
    distances = []
    for x in range(window_x1, window_x2):
        for y in range(window_y1, window_y2):
            distance = depth_frame.get_distance(x, y)
            if distance > 0:  # Ignore invalid distances
                distances.append(distance)

    if distances:
        return np.mean(distances)
    else:
        return 0  # Return 0 if no valid distances


prev_time = time.time()
try:
    while not rospy.is_shutdown():
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())
        color_image2 = color_image  ##출력용

        # Run YOLOv8 inference
        results = model(color_image, device=device, conf=0.5)
        
        # YOLO detection
        det= results[0]
        boxes2 = det.boxes
        masks2 = results_seg[0].masks

        person_detections= []
        weapon_boxes= []
        if boxes2 is not None:
            for i, box in enumerate(boxes2):
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_id = int(box.cls[0])
                conf = float(box.conf[0])

                cv2.rectangle(color_image2, (x1,y1), (x2,y2), (0,255,0),2)
                label = f"{model.names[class_id]}: {conf:.2f}"
                cv2.putText(color_image2, label,(x1,y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)

                # 세그 윤곽선(사람만)
                if masks2 is not None and class_id==0:
                    if i < len(masks2.data):
                        single_mask= masks2.data[i].cpu().numpy()
                        mask_bin= (single_mask>0.5).astype(np.uint8)
                        contours,_= cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        cv2.drawContours(color_image2, contours, -1, (0,255,255), 2)

                # DeepSORT
                if class_id==0:
                    w= x2 - x1
                    h= y2 - y1
                    person_detections.append(((x1,y1,w,h), conf,0))
                elif class_id in [1,2]:
                    weapon_boxes.append((x1,y1,x2,y2))

        # DeepSORT update
        tracks= tracker.update_tracks(person_detections, frame=color_image)
        tracked_boxes=[]
        for t in tracks:
            if not t.is_confirmed() or t.time_since_update>1:
                continue
            tid= t.track_id
            l,t_,r,b_ = map(int,t.to_ltrb())
            tracked_boxes.append((tid,l,t_,r,b_))
            cv2.putText(color_image2,f"ID:{tid}", (l-10,t_-10),
                        cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)

        # Arcface 로직
        time1 = time.time()
        max_arcface_frames=10 # id 별로 초기 10프레임만 얼굴 인식
    
        for (tid, px1, py1, px2, py2) in tracked_boxes:
            if tid not in track_me_status:
                track_me_status[tid]= False
            if tid not in track_arcface_count:
                track_arcface_count[tid]=0

            if track_me_status[tid]==False and track_arcface_count[tid]<max_arcface_frames:
                print(f"Running MobileFacenet for ID {tid} (Frame {track_arcface_count[tid]+1}/{MAX_ARCFACE_FRAMES})")
                track_arcface_count[tid]+=1

            
                # Arcface 임베딩 추출
                PAD=10
                sub_face= color_image[max(0,py1-PAD): py2+PAD, max(0,px1-PAD): px2+PAD]
                if sub_face.size==0:
                    continue

                emb_image = get_face_embedding(sub_face)
                if emb_image is not None:
                    same_person, sim = is_my_face(emb_image, my_face_embedding, threshold=0.4)
                    if same_person:
                        track_me_status[tid]=True


            # 시각화
            if track_me_status[tid]:
                text_arc= f"          Me(sim={sim:.2f})"
                color=(0,255,0)
                if tid in dangerous_ids:
                    dangerous_ids.remove(tid)
            else:
                text_arc= "           NotMe(sim={sim:.2f})"
                color=(0,0,255)
            cv2.putText(color_image2,text_arc,(px1, py1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,0.6,color,2)
        time2 = time.time()
        print("Arcface running time: ", time2-time1)

        # 무기 교차 & NotMe => dangerous
        for (tid, px1, py1, px2, py2) in tracked_boxes:
            if not track_me_status[tid]:
                pbox= (px1, py1, px2, py2)
                for wb in weapon_boxes:
                    if boxes_overlap(pbox, wb):
                        dangerous_ids.add(tid)
                        break

        # Dangerous => Mediapipe pose && PID
        for (tid, px1, py1, px2, py2) in tracked_boxes:
            if tid in dangerous_ids:    
                sub= color_image[py1:py2, px1:px2]
                if sub.size==0:
                    continue
                #Mediapipe
                c_rgb= cv2.cvtColor(sub, cv2.COLOR_BGR2RGB)
                pose_result= pose_danger.process(c_rgb)
                if pose_result.pose_landmarks:
                    lms= pose_result.pose_landmarks.landmark
                    sub_w= px2 - px1
                    sub_h= py2 - py1

                    left_shoulder_y= lms[LEFT_SHOULDER].y
                    right_shoulder_y= lms[RIGHT_SHOULDER].y
                    left_wrist_y= lms[LEFT_WRIST].y
                    right_wrist_y= lms[RIGHT_WRIST].y

                    la_up= (left_wrist_y< (left_shoulder_y-0.05))
                    ra_up= (right_wrist_y<(right_shoulder_y-0.05))
                    if la_up and ra_up:
                        a_text= "both arms up"
                    elif la_up:
                        a_text= "left arm up"
                    elif ra_up:
                        a_text= "right arm up"
                    else:
                        a_text= "do nothing"

                    for lm in lms:
                        cx= px1+int(lm.x*sub_w)
                        cy= py1+int(lm.y*sub_h)
                        cv2.circle(color_image2,(cx,cy),3,(0,255,255),-1)
                    cv2.putText(color_image2,f"Dangerous person: {a_text}",
                                (px1,py1+20),
                                cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                #PID
                box_width = px2 - px1  #add rotate codes to set person bounding box middle match camera middle PID
                box_height = py2 - py1
                if box_width > 10 and box_height > 10:
                    # Calculate the center 10x10 window's distance average
                    depth = get_center_window_distance(depth_frame, px1, py1, px2, py2, window_size=30)

                    # PID control for linear velocity (distance control)
                    error_linear = TARGET_DISTANCE - depth
                    velocity_linear, prev_error_linear, integral_linear = compute_pid(
                        error_linear, prev_error_linear, integral_linear, Kp_linear, Ki_linear, Kd_linear
                    )

                    # PID control for angular velocity (angle control)
                    image_center_x = color_image.shape[1] // 2
                    box_center_x = (px1 + px2) // 2
                    error_angular = image_center_x - box_center_x
                    angular_velocity, prev_error_angular, integral_angular = compute_pid(
                        error_angular, prev_error_angular, integral_angular, Kp_angular, Ki_angular, Kd_angular
                    )

                    # Limit velocity and angular velocity
                    velocity_linear = max(min(velocity_linear, 0.4), -0.4)  # Limit linear speed
                    angular_velocity = max(min(angular_velocity, 0.4), -0.4)  # Limit angular speed

                    # Publish velocity command
                    cmd_vel = Twist()
                    cmd_vel.linear.x = velocity_linear
                    cmd_vel.angular.z = angular_velocity
                    cmd_vel_pub.publish(cmd_vel)

                    # Draw bounding box and display distance
                    #cv2.rectangle(color_image2, (x1, y1), (x2, y2), (0, 255, 0), 2)   #YOLO단계에서 박스침
                    cv2.putText(color_image2, f"{depth:.2f}m", (px1, py1 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    # If the box is too small, stop moving
                    cmd_vel_pub.publish(Twist())  # Stop robot
                        
                break     # Pose detection, PID for 1 person

        # FPS 표시
        now= time.time()
        fps= 1.0/(now - prev_time)
        prev_time= now
        cv2.putText(color_image2,f"FPS:{fps:.2f}",(10,30),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)   

        # Show detection
        cv2.imshow("Kobuki Person Following", color_image2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    cmd_vel_pub.publish(Twist())  # Stop robot before exiting